In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer,SincBatchLeaky
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss,LayerNorm,BatchNorm1d
from torch.utils.data import DataLoader
from torch.optim import Adam

In [164]:
## change ##
batch_size = 16
clip = .1
block = ResidualBlock
normalFun = normalize2
name = 'raw_sincNet_attention'

In [3]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalFun(train[:,0])
# without tanh is worse-off

Prepare data

In [7]:
# train_gen = SequenceGen(train)
# train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

# val_gen = SequenceGen(train,False)
# val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

In [5]:
train_gen = SequenceGenSample(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSample(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

In [69]:
model(x).shape

torch.Size([16, 128, 167])

Build model

In [167]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadedAttention(2,size,dropout)
        self.feed_forward = PositionwiseFeedForward(size,size,dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(size, dropout),SublayerConnection(size, dropout)])
        self.size = size

    def forward(self, x, mask=None):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)   

In [190]:
model = Sequential(   LayerNorm(37500),
                      Dropout(0.1),
                      SincBatchLeaky(1,128,256,sample_rate=1000000,stride=128),
                      Dropout(0.1),
                      LambdaLayer(lambda x:x.transpose(1,2)),
                      EncoderLayer(128,0.5),
                      Linear(128,1,bias=False),
                      Dropout(0.1),                 
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),                 
                      biasLayer((1,)),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [191]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [192]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [163]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.68786307825492, val_loss:2.562455177307129
epoch:1, train_loss:2.2781395877783117, val_loss:2.5355451107025146
epoch:2, train_loss:2.2009986696334987, val_loss:2.476217031478882
epoch:3, train_loss:2.1581294926313253, val_loss:2.4494194984436035
epoch:4, train_loss:2.1159924664176426, val_loss:2.496201276779175
Training completed in 15.810860872268677s


Submission

In [166]:
make_submission_sample(name,model,normalFun,batch_size)